# Verify the cost model fidelity here
- notice: the code is tested on A800, thus didn't distributed
    - for larger model 66b, single node may not be able to hold, refer to the shard in QLLM to test the memory size to create shards among nodes.
- if tested on smaller GPU with lower CPU capability, pls check the examples to load in numpy

In [48]:
import torch
# sample batch size from [4, 8, 16]
# sample prompt length from [128, 512]
# sample maximum generated token from [100, 200]
# sample precision from [3, 4, '8:tc-li', 16]
batch_sizes = [2, 4, 8]
prompt_lengths = [128, 512]
generated_tokens = [100, 200]
precision_candidates = [3, 4, '8:tc-li', 16]
# set cuda device
cuda_device_num = 3
# torch.cuda.set_device(cuda_device_num)
device = torch.device(f"cuda:{cuda_device_num}" if torch.cuda.is_available() else "cpu")

In [49]:
from qllm.models import create_model_config
# sample cases
# model
models = [
    ['bloom', '560m'],
    ['bloom', '1b7'],
    ['opt', '13b'],
    ['opt', '30b'],
    ['opt', '66b']
]
model_configs = [
    create_model_config(model_name, model_size) for (model_name, model_size) in models
]

In [50]:
import random
from qllm.utils import create_single_node_sharding_strategies_with_precision_specs
def generate_configurations(model_configs):
    configs = {}
    for idx, config in enumerate(model_configs):
        # get layer number
        if hasattr(config, 'n_layer'):
            dec_layers = config.n_layer
        elif hasattr(config,"num_hidden_layers"):
            dec_layers = config.num_hidden_layers
        else:
            raise ValueError("Not implemented")
        batch_size = random.choice(batch_sizes)
        prompt_length = random.choice(prompt_lengths)
        generated_token = random.choice(generated_tokens)
        layer_precs = [
            random.choice(precision_candidates) for _ in range(dec_layers)
        ]
        layer_precs = create_single_node_sharding_strategies_with_precision_specs(dec_layers, layer_precs)
        
        configs[idx] = [batch_size, prompt_length, generated_token, layer_precs]
    
    return configs


In [51]:
# sample from 
mem_sample_configs = generate_configurations(model_configs)

In [52]:
# check the generated configs
mem_sample_configs

{0: [8,
  512,
  200,
  {0: {0: {'shard': [0, 1], 'bits': [3, 3]},
    1: {'shard': [0, 1], 'bits': [16, 16]},
    2: {'shard': [0, 1], 'bits': ['8:tc-li', '8:tc-li']},
    3: {'shard': [0, 1], 'bits': [4, 4]},
    4: {'shard': [0, 1], 'bits': [4, 4]},
    5: {'shard': [0, 1], 'bits': [4, 4]},
    6: {'shard': [0, 1], 'bits': ['8:tc-li', '8:tc-li']},
    7: {'shard': [0, 1], 'bits': [3, 3]},
    8: {'shard': [0, 1], 'bits': [16, 16]},
    9: {'shard': [0, 1], 'bits': [16, 16]},
    10: {'shard': [0, 1], 'bits': [3, 3]},
    11: {'shard': [0, 1], 'bits': [3, 3]},
    12: {'shard': [0, 1], 'bits': [16, 16]},
    13: {'shard': [0, 1], 'bits': [16, 16]},
    14: {'shard': [0, 1], 'bits': ['8:tc-li', '8:tc-li']},
    15: {'shard': [0, 1], 'bits': [3, 3]},
    16: {'shard': [0, 1], 'bits': [16, 16]},
    17: {'shard': [0, 1], 'bits': [4, 4]},
    18: {'shard': [0, 1], 'bits': [3, 3]},
    19: {'shard': [0, 1], 'bits': ['8:tc-li', '8:tc-li']},
    20: {'shard': [0, 1], 'bits': [16, 16]},
    

In [57]:
# check the memory occupation
from qllm.models import create_empty_model, qllm_load_pretrained_from_size
from qllm.utils import get_model_size_cuda, ModelMemEstimator, get_iter_variable_size
import lptorch
import os 
def check_mem_occupation(model_name_size_pair, model_config, mem_sample_config):
    # create empty model is enough to estimate.
    # load weight if you want.
    model_name, model_size = model_name_size_pair
    model = create_empty_model(model_name, model_size)
#     model, tokenizer, key = qllm_load_pretrained_from_size(model_name, model_size)
    caliber = lptorch.inner_caliber
    caliber.set_model(model)
    caliber.set_fake()
    caliber.load_fake_calib_data(f'./fake_calib_{model_name}_{model_size}.pkl')
    
    batch_size, prompt_length, generated_token, sharding_strategy = mem_sample_config
    # shard model
    model_pre_and_post = model._pure_pre_and_post()
    model = model.float() # some op need fp32 to make quantization
    model = model.shard_model(sharding_strategy, 0) # generated shard strategy is single node
    
    # move model to cuda
    model_pre_and_post = model_pre_and_post.cuda()
    model.decoder_layers_to_device(device)
    # init kv
    model.init_kv_cache(batch_size, prompt_length, generated_token, request_id=1)

    # get estimated size
    if model_name == 'bloom':
        h1 = model_config.hidden_size
        vocab_size = model_config.vocab_size
        max_position_embeddings = 0
        word_embed_proj_dim = h1
        h2 = h1 * 4
        
    elif model_name == 'opt':
        h1 = model_config.hidden_size
        h2 = model_config.ffn_dim
        vocab_size = model_config.vocab_size
        max_position_embeddings = model_config.max_position_embeddings
        word_embed_proj_dim = model_config.word_embed_proj_dim
        
    model_mem_estimator = ModelMemEstimator(h1, h2, batch_size, prompt_length, generated_token, \
                                            vocab_size, max_position_embeddings, word_embed_proj_dim)
    
    # comparison
    overall_real_mem = 0
    overall_est_mem = 0
    # data rank
    model_pre_post_real = get_model_size_cuda(model_pre_and_post, 'MB')[0] + get_model_size_cuda(model_pre_and_post.lm_head, 'MB')[0]
    model_pre_post_estimate = model_mem_estimator.calculate_prepost_mem(unit='MB')[0]
    print(f"Model pre and post size: {model_pre_post_real} MB")
    print(f"Est Model pre and post size: {model_pre_post_estimate} MB")
    overall_real_mem += model_pre_post_real
    overall_est_mem += model_pre_post_estimate
    # model
    model_mem_real = get_model_size_cuda(model, 'MB')[0]
    model_mem_est = model_mem_estimator.calculate_model_occupation_of_partition(sharding_strategy[0], unit='MB')[0]
    print(f"Model size: {model_mem_real} MB")
    print(f"Estimated Model {model_mem_est} MB")
    overall_real_mem += model_mem_real
    overall_est_mem += model_mem_est
    # KV size
    request_num = 1 # allow more request nums 
    kv_mem_est = kv_mem_real = 0
    if model_name == 'bloom':
        kv_mem_real = get_iter_variable_size(model.transformer.get_all_kv_cache_dict(), unit='MB')
        kv_mem_est = request_num * model_mem_estimator.calculate_kv_occupation_of_partition(sharding_strategy[0], 'MB')[0]

    elif model_name == 'opt':
        kv_mem_real = get_iter_variable_size(model.model.decoder.get_all_kv_cache_dict(), unit='MB')
        kv_mem_est = request_num * model_mem_estimator.calculate_kv_occupation_of_partition(sharding_strategy[0], 'MB')[0]
    overall_real_mem += kv_mem_real
    overall_est_mem += kv_mem_est
    print(f"Model KV size: {kv_mem_real}MB")
    print(f"Estimated Model KV: {kv_mem_est}MB")
    print(f"Overall mem: Real {overall_real_mem} Vs. Est {overall_est_mem}")

In [55]:
def check_mem_est(check_idx):
    model_name_size_pair = models[check_idx]
    model_config = model_configs[check_idx]
    mem_sample_config = mem_sample_configs[check_idx]
    check_mem_occupation(model_name_size_pair, model_config, mem_sample_config)

In [21]:
# Generate some fake calibaration data 
!python3 fake_calib_sample.py --model-name bloom --model-size 560m
!python3 fake_calib_sample.py --model-name bloom --model-size 1b7
!python3 fake_calib_sample.py --model-name opt --model-size 13b
!python3 fake_calib_sample.py --model-name opt --model-size 30b
!python3 fake_calib_sample.py --model-name opt --model-size 66b


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following di


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following di

In [58]:
# select the check_idx
check_idx = 0
check_mem_est(check_idx)

Model pre and post size: 980.0078125 MB
Est Model pre and post size: 980.00390625 MB
Model size: 320.98809814453125 MB
Estimated Model 319.59375 MB
Model KV size: 534.0MB
Estimated Model KV: 534.0MB
Overall mem: Real 1834.9959106445312 Vs. Est 1833.59765625


In [59]:
check_idx = 1
check_mem_est(check_idx)

Model pre and post size: 1960.015625 MB
Est Model pre and post size: 1960.0078125 MB
Model size: 1011.1935424804688 MB
Estimated Model 1008.1875 MB
Model KV size: 459.0MB
Estimated Model KV: 459.0MB
Overall mem: Real 3430.2091674804688 Vs. Est 3427.1953125
